In [ ]:
#    Copyright 2020 Arkadip Bhattacharya

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

In [1]:
%matplotlib qt

%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import dataloader
from sklearn.model_selection import train_test_split
from preprocessing import Normalize_df, WindSpeedDataset, ComposeTransform, ToTensor

In [2]:
dataset = Normalize_df(pd.read_csv('./dataset-daily.csv'))
dataset.head()

,time,air_temperature_mean,pressure,wind_direction,wind_speed
0,0.000000,0.370203,0.103164,0.732591,0.625000
1,0.000011,0.322799,0.268912,0.838440,0.354167
2,0.000022,0.302483,0.709078,0.988858,0.260417
3,0.000033,0.246050,0.850758,0.239554,0.093750
4,0.000044,0.194131,0.827372,0.345404,0.291667


In [3]:
trainset, testset = train_test_split(dataset, test_size = 0.1)
trainset, valset = train_test_split(trainset, test_size = 0.1)

In [4]:
trainset.iloc[0]

time                    0.117590
air_temperature_mean    0.751693
pressure                0.616231
wind_direction          0.030641
wind_speed              0.114583
Name: 594, dtype: float64

In [5]:
train_dataset = WindSpeedDataset(trainset,transform=ComposeTransform([ToTensor()]))
test_dataset = WindSpeedDataset(testset, transform=ComposeTransform([ToTensor()]))
val_dataset = WindSpeedDataset(valset, transform=ComposeTransform([ToTensor()]))

In [6]:
train_dataset[0]

(tensor([0.7517, 0.6162, 0.0306], dtype=torch.float64),
 tensor([0.1146], dtype=torch.float64))

In [7]:
batch_size = 128

trainloader = dataloader.DataLoader(train_dataset, batch_size, shuffle = True)
valloader = dataloader.DataLoader(val_dataset, batch_size, shuffle = False)
testloader = dataloader.DataLoader(test_dataset, batch_size, shuffle = False)

In [8]:
cuda = torch.cuda.is_available()
device = torch.device('cuda') if cuda else torch.device('cpu')
if cuda:
    print("Device Count:", torch.cuda.device_count())
    print("Device:", torch.cuda.get_device_name())
    print("Device Capability:", torch.cuda.get_device_capability())

Device Count: 1
Device: GeForce 940MX
Device Capability: (5, 0)


In [9]:
from model import Model
model = Model()
model.to(device)
print(model)

Model(
  (fc1): Linear(in_features=3, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=1, bias=True)
)


In [10]:
from torch.optim import SGD
from torch.nn import MSELoss

trainlosses, testlosses = model.fit(trainloader = trainloader,
                                     validationloader = valloader,
                                     loss = MSELoss,
                                     optim = SGD,
                                     lr=0.01,
                                     epochs = 5,
                                     val_per_batch = 20,
                                     cuda = cuda)

Epoch: 1 Batch: 0 out of 24 Training Loss: 0.00472859541575114 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 1 out of 24 Training Loss: 0.10025831560293834 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 2 out of 24 Training Loss: 0.1970897341767947 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 3 out of 24 Training Loss: 0.26462064186731976 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 4 out of 24 Training Loss: 0.3408873478571574 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 5 out of 24 Training Loss: 0.4142983208100001 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 6 out of 24 Training Loss: 0.46680030102531117 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 7 out of 24 Training Loss: 0.5251568319896858 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 8 out of 24 Training Loss: 0.5671377244095008 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 9 out of 24 Training Loss: 0.6168543348709743 Test Loss: 0.10324156284332275
Epoch: 1 Batch: 10 out of 24 Training Loss: 0.6485202858845394 Tes

Epoch: 4 Batch: 21 out of 24 Training Loss: 0.04083485705713327 Test Loss: 0.02559218741953373
Epoch: 4 Batch: 22 out of 24 Training Loss: 0.06286936802724893 Test Loss: 0.02559218741953373
Epoch: 4 Batch: 23 out of 24 Training Loss: 0.08609573906222875 Test Loss: 0.02559218741953373
Epoch: 5 Batch: 0 out of 24 Training Loss: 0.0009884803245464961 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 1 out of 24 Training Loss: 0.022602616498867672 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 2 out of 24 Training Loss: 0.04192670310537021 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 3 out of 24 Training Loss: 0.06283152351776759 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 4 out of 24 Training Loss: 0.0902324231962363 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 5 out of 24 Training Loss: 0.1116514119009177 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 6 out of 24 Training Loss: 0.1325323699663083 Test Loss: 0.02551831987996896
Epoch: 5 Batch: 7 out of 24 Training Loss: 0.157237128044

In [11]:
plt.plot(trainlosses, label = 'Training Losses')
plt.plot(testlosses, label = 'Testing Losses')
plt.ylabel('Losses')
plt.xlabel('Batch * Epoch')
plt.legend()
plt.title('Losses Graph')
plt.show()

In [12]:
result, expected = model.test(testloader, cuda=cuda)

In [13]:
val_from = 0
val_to = 20

plt.plot(result[val_from:val_to], label = 'Result')
plt.plot(expected[val_from:val_to], label = 'Expected')
plt.ylabel('Wind Speed (Normalized)')
plt.xticks(range(val_from, val_to))
plt.xlabel('Time Range')
plt.legend()
plt.title('Model Output Validation')
plt.show()

In [14]:
model.save('v1', save_optim=True, cuda=cuda)

v1  - dir Already exists
Model saved in v1
Optimizer saved in v1
